In [66]:
import torch
import torch.nn as nn
import torch.optim as  optim
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [67]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [68]:
df=pd.read_csv('riceClassification.csv')
df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [69]:
df.dropna(inplace=True)
df.drop(['id'],axis=1,inplace=True)
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [70]:
print(df["Class"].value_counts())


Class
1    9985
0    8200
Name: count, dtype: int64


In [71]:
original_df=df.copy()

for column in df.columns:
  df[column]=df[column]/df[column].abs().max()

df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [72]:
X=np.array(df.iloc[:,:-1])
y=np.array(df.iloc[:,-1])

In [73]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [75]:
class dataset(Dataset):
  def __init__(self,X,Y):
    self.X=torch.tensor(X,dtype=torch.float32).to(device)
    self.Y=torch.tensor(Y,dtype=torch.long).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self,idx):
    return self.X[idx],self.Y[idx]


In [76]:
train_dataset=dataset(X_train,y_train)
test_dataset=dataset(X_test,y_test)

In [77]:
class MyNN(nn.Module):

  def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate):

    super().__init__()

    layers = []
    current_input_dim = input_dim # Use a temporary variable to track the evolving input dimension

    for i in range(num_hidden_layers):

      layers.append(nn.Linear(current_input_dim, neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.Sigmoid())
      layers.append(nn.Dropout(dropout_rate))
      current_input_dim = neurons_per_layer # Update input dimension for the next layer

    # Final layer should output a single value for binary classification logits
    layers.append(nn.Linear(neurons_per_layer, 1)) # Changed output dimension to 1

    self.model = nn.Sequential(*layers)

  def forward(self, x):

    return self.model(x)

In [78]:
# objective function
def objective(trial):

  # next hyperparameter values from the search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 10)
  neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  epochs = trial.suggest_int("epochs", 10, 100, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  # model init
  # Correct the input_dim to match the number of features in X
  input_dim = X.shape[1]
  # Set output_dim to 1 for binary classification with a single output neuron
  output_dim = 1

  model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
  model.to(device)

  # optimizer selection
  # Use BCEWithLogitsLoss for stability with sigmoid output and raw scores
  criterion = nn.BCEWithLogitsLoss()

  # Initialize the optimizer based on the suggested optimizer_name
  if optimizer_name == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else: # RMSprop
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  # training loop

  for epoch in range(epochs):

    model.train() # Set model to training mode
    running_loss = 0.0
    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      # For BCEWithLogitsLoss, target should be float and have shape (batch_size, 1)
      batch_labels = batch_labels.float().unsqueeze(1)


      # forward pass
      outputs = model(batch_features) # outputs are logits (batch_size, 1)

      # calculate loss
      loss = criterion(outputs, batch_labels)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()

      running_loss += loss.item()

    # Optional: print training loss per epoch
    # print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")


  # evaluation
  model.eval() # Set model to evaluation mode
  # evaluation on test data
  total = 0
  correct = 0

  with torch.no_grad():

    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      outputs = model(batch_features) # outputs are logits (batch_size, 1)

      # For binary classification with a single logit output:
      # Apply sigmoid to get probabilities
      probabilities = torch.sigmoid(outputs)
      # Threshold probabilities at 0.5 to get predicted class (0 or 1)
      predicted = (probabilities > 0.5).long() # Convert boolean to Long tensor

      total = total + batch_labels.shape[0]

      # Ensure batch_labels has the same shape as predicted (batch_size, 1) for comparison
      batch_labels = batch_labels.unsqueeze(1)


      correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total

  return accuracy
  # The print statement after return will not be executed
  # print(accuracy)

In [60]:
!pip install optuna

In [79]:
import optuna

study = optuna.create_study(direction='maximize')

[I 2025-05-17 11:50:57,884] A new study created in memory with name: no-name-3d0c6702-f41e-4f3f-adb4-c9fdbf4dadd7


In [80]:
study.optimize(objective, n_trials=10)

[I 2025-05-17 11:51:11,036] Trial 0 finished with value: 0.9879032258064516 and parameters: {'num_hidden_layers': 6, 'neurons_per_layer': 128, 'epochs': 60, 'learning_rate': 0.058989236284579194, 'dropout_rate': 0.2, 'batch_size': 128, 'optimizer': 'SGD', 'weight_decay': 0.0001112954036434219}. Best is trial 0 with value: 0.9879032258064516.
[I 2025-05-17 11:51:19,025] Trial 1 finished with value: 0.5515029325513197 and parameters: {'num_hidden_layers': 10, 'neurons_per_layer': 128, 'epochs': 30, 'learning_rate': 0.00013006410822608786, 'dropout_rate': 0.30000000000000004, 'batch_size': 128, 'optimizer': 'SGD', 'weight_decay': 1.7526627934291645e-05}. Best is trial 0 with value: 0.9879032258064516.
[I 2025-05-17 11:52:42,252] Trial 2 finished with value: 0.5515029325513197 and parameters: {'num_hidden_layers': 9, 'neurons_per_layer': 40, 'epochs': 50, 'learning_rate': 0.02596114526360221, 'dropout_rate': 0.5, 'batch_size': 16, 'optimizer': 'SGD', 'weight_decay': 8.603114262107424e-05}.

In [81]:
study.best_params

{'num_hidden_layers': 5,
 'neurons_per_layer': 104,
 'epochs': 30,
 'learning_rate': 0.006648344728236754,
 'dropout_rate': 0.30000000000000004,
 'batch_size': 64,
 'optimizer': 'Adam',
 'weight_decay': 0.00012476084968130006}

In [82]:
study.best_value

0.9897360703812317